<a href="https://colab.research.google.com/github/summermccune/Tokenization-Testing-for-Malware-Data/blob/main/Word2Vec/Word2Vec%26SVM_bigrams%26trigrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
from gensim.utils import simple_preprocess
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import glob
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import trigrams
from nltk.util import bigrams
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Currently splitting into bigrams and trigrams by each sample, not within each sample. So, one sample would be split into trigrams and the other into bigrams and so on until it goes through all the samples

In [13]:
def read_files(malwarefolder, malwareType):
  count = 0
  for sample in malwarefolder:
    count += 1
    with open(sample, 'r') as f:
      data = f.read().split()
      if count % 2 == 0:
        #make opcodes into trigrams
        trigrms = list(trigrams(data))
        #make the trigrams each their own string
        trigrms = [f"{first} {second} {third}" for first, second, third in trigrms]
        data = trigrms
      else:
        #make opcodes into bigrams
        bigrms = list(bigrams(data))
        #make the bigrams each their own string
        bigrms = [f"{first} {second}" for first, second in bigrms]
        data = bigrms
    #append sample to opcodes list in df
    df.loc[len(df)] = [data, malwareType]

In [ ]:
#creating dataframe for samples and label
df = pd.DataFrame(columns = ['opcodes','label'])

#specify paths for each malware
winwebsec = glob.glob("/content/drive/MyDrive/Data/malware2/winwebsec/*.txt")
zbot = glob.glob("/content/drive/MyDrive/Data/malware2/zbot/*.txt")
zeroaccess = glob.glob("/content/drive/MyDrive/Data/malware2/zeroaccess/*.txt")

#read files
read_files(winwebsec, 0)
read_files(zbot, 1)
read_files(zeroaccess, 2)

print(df.shape)
df.head()

In [15]:
#word2vec
model = Word2Vec(df['opcodes'], min_count=1, vector_size=100)

In [ ]:
#SVM
X = np.array([np.mean([model.wv[word] for word in text if word in model.wv]
                        , axis=0) for text in df['opcodes']])
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
svm = SVC(kernel='rbf')
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

#accuracy and conf matrix
print("Accuracy:", np.mean(y_pred == y_test))
print(confusion_matrix(y_test, y_pred))